In [1]:
from skimage import io, transform
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
import sys
import pandas
from conv_model import SimpleConvNet
from conv_model import ConvNetWithPooling
from dataLoader import OrganoidDataset
%load_ext autoreload
%autoreload 2

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

params = {'batch_size': 100, # low for testing
  'shuffle': True, 'num_workers' : 1}

max_epochs = 200

#path = '/Users/Daley/Teaching/CS231N/CS231Nproject/CS231n_Tim_Shan_example_data/' # need to change
path = '../data/CS231n_Tim_Shan_example_data/'

well_descriptions = pandas.read_csv('processed_well_descriptions.txt', sep='\t', header=0)
#sizes = well_descriptions['mw_area shape'].tolist()
day1wells = well_descriptions[(well_descriptions['day'] == 1)]
day13wells = well_descriptions[(well_descriptions['day'] == 13)]
finalSizes = day13wells['normalized hyst2 area'].values

well_labels = []
for i in range(4800):
  i2str = str(i)
  if len(i2str) == 1:
    i2str = '000' + i2str
  if len(i2str) == 2:
    i2str = '00' + i2str
  if len(i2str) == 3:
    i2str = '0' + i2str
  well_labels.append(i2str)

initial_train_set = OrganoidDataset(path2files = path, well_labels = well_labels[0:1000], day_label_X = ['01']*1000, sizes = finalSizes[0:1000])
training_generator = data.DataLoader(initial_train_set, **params)

In [ ]:
model = nn.Sequential(
    nn.Conv2d(in_channels=in_channel,out_channels=channel_1,kernel_size=5,padding=2,bias=True),
    nn.ReLU(),
    nn.Conv2d(in_channels=channel_1,out_channels=channel_2,kernel_size=3,padding=1,bias=True),
    nn.ReLU(),   
    Flatten(),
    nn.Linear(channel_2*image_size*image_size, output_channel,bias=True),
)

In [14]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

test_flatten()
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

Before flattening:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])
After flattening:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


In [18]:
in_channels = 1
out_size = 1
in_channels = 1 
layer1channels = 16 
layer2channels = 16 
out_size = 1
model = nn.Sequential(
    nn.Conv2d(in_channels, layer1channels,kernel_size=5, stride=1, padding=2),
    nn.BatchNorm2d(layer1channels),
    nn.ReLU(),
    
    nn.Conv2d(layer1channels, layer2channels,
                                          kernel_size=5, stride=1, padding=2),
    nn.BatchNorm2d(layer2channels),
    nn.ReLU(),
    
    nn.Conv2d(layer2channels, layer2channels,kernel_size=5, stride=1, padding=2),
    nn.BatchNorm2d(layer2channels),
    nn.ReLU(),
    nn.MaxPool2d(2),
    
    Flatten(),
    nn.Linear(in_features = 8*193*193, out_features = out_size),
)

In [ ]:
#     self.layer1 = nn.Sequential(nn.Conv2d(in_channels, layer1channels,
#                                           kernel_size=5, stride=1, padding=2),
#                                 nn.BatchNorm2d(layer1channels),
#                                 nn.ReLU())
#     self.layer2 = nn.Sequential(nn.Conv2d(layer1channels, layer2channels,
#                                           kernel_size=5, stride=1, padding=2),
#                                 nn.BatchNorm2d(layer2channels),
#                                 nn.ReLU())
#     self.layer3 = nn.Sequential(nn.Conv2d(layer2channels, layer2channels,
#                                           kernel_size=5, stride=1, padding=2),
#                                 nn.BatchNorm2d(layer2channels),
#                                 nn.ReLU(),
#                                 nn.MaxPool2d(2),
#                                 )
#     self.fc = nn.Linear(in_features = 193*193*layer2channels, out_features = out_size)

In [12]:

#model = SimpleConvNet(in_channels = in_channels, out_size = out_size).to(device)
#model = ConvNetWithPooling(in_channels = in_channels, out_size = out_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.999), eps=10**-8, weight_decay=0)
loss = nn.MSELoss()
train_error_array = np.zeros(max_epochs)




In [19]:
for epoch in range(max_epochs):
  # Training
  print(epoch)
  optimizer.zero_grad()
  totalbatchMSE = 0.0
  for local_X, local_Y in training_generator:
    local_X, local_Y = local_X.to(device), local_Y.to(device)
    Y_hat = model.forward(local_X)
    train_error = loss(Y_hat, local_Y)
    train_error.backward()
    optimizer.step()
    model.eval() # set evaluation mode
    Y_hat = model.forward(local_X)
    train_error = loss(Y_hat, local_Y).item()
    totalbatchMSE = totalbatchMSE + params['batch_size']*train_error/4800 # rescale train_error, since MSE = \sum sqrt(|Y - hat(Y)|^2) / batch_size
    train_error_array[epoch] = totalbatchMSE
#     if t % print_every == 0:
#         print(totalbatchMSE)
    #np.savetxt(fname = "train_error_array_lr0.01_3convLayers.txt", X = train_error_array[range(epoch)])

0


OSError: [Errno 12] Cannot allocate memory